In [ ]:
!pip3 install thirdai
!pip3 install pandas # Needed by the datasets module to convert datasets to CSV format
!pip3 install numpy

In [ ]:
import utils

train_filename, test_filename, inference_batch = utils.download_clinc()

In [ ]:
from thirdai import bolt

model = bolt.UniversalDeepTransformer(
    data_types={
        "text": bolt.types.text(),
        "category": bolt.types.categorical(n_unique_classes=151)
    },
    target="category"
)

In [ ]:
train_config = (bolt.TrainConfig(epochs=5, learning_rate=0.01)
                    .with_metrics(["categorical_accuracy"]))

model.train(train_filename, train_config)

eval_config = (bolt.EvalConfig()
                   .with_metrics(["categorical_accuracy"]))

model.evaluate(test_filename, eval_config);

In [ ]:
print(inference_batch)

In [ ]:
import numpy as np

prediction = model.predict(inference_batch[0])
class_name = model.class_name(np.argmax(prediction))
print("Single Prediction Result")
print("Input:", inference_batch[0], "Prediction:", class_name, "\n")

prediction_batch = model.predict_batch(inference_batch)
class_ids = np.argmax(prediction_batch, axis=1)
class_names = [model.class_name(class_id) for class_id in class_ids]
print("Batch Prediction Results")
for input_sample, class_name in zip(inference_batch, class_names):
    print("Input:", input_sample, "Prediction:", class_name)